In [1]:
import os
os.getcwd()

'/Users/nithinpradeep/AV/Projects/AI-vs-Real-Art-classification-end-to-end/research'

In [2]:
os.chdir('../')
os.getcwd()

'/Users/nithinpradeep/AV/Projects/AI-vs-Real-Art-classification-end-to-end'

###  Step 1: Create Entity

In [3]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_dense_neurons: int

### Step 2: Configuration

In [4]:
from AI_Real_Classifier.constants import *
from AI_Real_Classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #this will create the artifacts folder

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_dense_neurons= self.params.DENSE_NEURONS
            
        )

        return prepare_base_model_config

### Step 3: Create component

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config= config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
        

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, dense_neurons ):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till > 0:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        x1= tf.keras.layers.Dense(units= dense_neurons, activation= 'relu', name= 'fc1', kernel_regularizer=tf.keras.regularizers.l1(0.01))(flatten_in)
        x2= tf.keras.layers.Dropout(0.5, name="Dropout")(x1)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(x2)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model


    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=False,
            freeze_till=15,
            learning_rate=self.config.params_learning_rate,
            dense_neurons= self.config.params_dense_neurons
        )
        
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)



    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

### Step 4: Pipeline

In [15]:
try: 
    config= ConfigurationManager() #initialize the configuration through ConfigurationManager class
    prepare_base_model_config= config.get_prepare_base_model_config() ##get the prepare base model config of the config object
    prepare_base_model= PrepareBaseModel(config= prepare_base_model_config) #pass the prepare base model config to the prepare base model component
    prepare_base_model.get_base_model() # get the base model from the component
    prepare_base_model.update_base_model() #update the base model

except Exception as e:
    raise e

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/opt/anaconda3/envs/aireal/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
model= tf.keras.models.load_model('/Users/nithinpradeep/AV/Projects/AI-vs-Real-Art-classification-end-to-end/artifacts/prepare_base_model/base_model_updated.h5')
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [17]:
for layers in model.layers:
    print(layers, layers.trainable)

<keras.src.engine.input_layer.InputLayer object at 0x2a944ba60> True
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2a944bdf0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2a944b2e0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x2ccc4e5b0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2ccc7db20> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2a944baf0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x2ccca6430> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2ccca6e50> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2ccca6f40> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2cccb1550> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x2cccb1b20> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2cccb6400> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x2cccb63d0> False
<keras.src